### Create the environment
To do so, we select an API endpoint and collect a token from our configuration. <br>

In [1]:
eosc_perf_api="https://performance.services.fedcloud.eu/api/v1"

### Configure the arguments and terms for your search

#### Search for the benchmark id that produced our result
You can get a list of all available benchmarks using GET /benchmarks

In [2]:
benchmarks=$(curl -X 'GET' "$eosc_perf_api/benchmarks?docker_image=deephdc/deep-oc-benchmarks_cnn&docker_tag=benchmark")
benchmark=$(echo $benchmarks | jq '.items[0]')
echo $benchmark | jq '.json_schema = "..."'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1547  100  1547    0     0   2918      0 --:--:-- --:--:-- --:--:--  2918
{
  "description": "tf_cnn_benchmarks accessed via DEEP API. Several Convolutional Neural Networks are executed to evaluate the performance.",
  "docker_image": "deephdc/deep-oc-benchmarks_cnn",
  "docker_tag": "benchmark",
  "id": "5a0f5641-01ae-4f48-b5d1-84cd9c87dfac",
  "json_schema": "...",
  "upload_datetime": "2021-10-13T15:07:45.672697"
}


In [3]:
benchmark_id=$(echo $benchmark | jq -r '.id')
curl -X 'GET' "$eosc_perf_api/results?benchmark_id=$benchmark_id" \
  -H 'accept: application/json' | jq '.items'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 43513  100 43513    0     0  78685      0 --:--:-- --:--:-- --:--:-- 78685
[
  {
    "benchmark": {
      "description": "tf_cnn_benchmarks accessed via DEEP API. Several Convolutional Neural Networks are executed to evaluate the performance.",
      "docker_image": "deephdc/deep-oc-benchmarks_cnn",
      "docker_tag": "benchmark",
      "id": "5a0f5641-01ae-4f48-b5d1-84cd9c87dfac",
      "json_schema": {
        "$schema": "https://json-schema.org/draft/2020-12/schema",
        "properties": {
          "benchmark": {
            "properties": {
              "dataset": {
                "suggestToUser": true,
                "type": "string"
              },
              "flavor": {
                "suggestToUser": true,
                "type": "string"
              },
              "version": {
                "type": "st

      "global_start_time": "2021-02-23T11:02:35.696854Z",
      "machine_config": {
        "cpu_info": {
          "cpu_info": "AMD EPYC 7742 64-Core Processor",
          "mhz_per_cpu": 3317.532,
          "num_cores": 256
        },
        "gpu_info": {
          "count": 8,
          "memory": 39420913408,
          "model": "A100-SXM4-40GB"
        },
        "memory_available": 1060777299968,
        "memory_total": 1082004615168
      },
      "training": {
        "allow_growth": true,
        "data_format": "NCHW",
        "device": "gpu",
        "googlenet": {
          "average_examples_per_sec": 1684.1920130854724,
          "batch_size": 960,
          "batch_size_per_device": 960,
          "execution_time_sec": 56.42832899093628,
          "num_batches": 100,
          "num_epochs": 0
        },
        "inception3": {
          "average_examples_per_sec": 481.3027947576969,
          "batch_size": 240,
          "batch_size_per_device": 240,
          "execution_time_

                  },
                  "mhz_per_cpu": {
                    "type": "number"
                  },
                  "num_cores": {
                    "type": "number"
                  }
                },
                "type": "object"
              },
              "gpu_info": {
                "properties": {
                  "count": {
                    "type": "number"
                  },
                  "model": {
                    "type": "string"
                  }
                },
                "type": "object"
              },
              "memory_available": {
                "type": "number"
              },
              "memory_total": {
                "type": "number"
              }
            },
            "type": "object"
          },
          "training": {
            "properties": {
              "device": {
                "suggestToUser": true,
                "type": "string"
              },
              "models": {
        

          "batch_size": 240,
          "batch_size_per_device": 240,
          "execution_time_sec": 40.239657163619995,
          "num_batches": 100,
          "num_epochs": 0
        },
        "score": 2936.8554382911675,
        "use_fp16": false,
        "variable_update": "parameter_server",
        "vgg16": {
          "average_examples_per_sec": 309.50671192646723,
          "batch_size": 160,
          "batch_size_per_device": 160,
          "execution_time_sec": 51.15352201461792,
          "num_batches": 100,
          "num_epochs": 0
        }
      }
    },
    "site": {
      "address": "https://wiki.scc.kit.edu/hpc/index.php?title=Category:ForHLR",
      "description": "Baden-Württemberg's general purpose tier 2 high performance computing (HPC) cluster",
      "id": "27ee87ff-a9aa-4486-891a-aff8164a9cab",
      "name": "ForHLR2",
      "upload_datetime": "2021-10-13T15:11:17.437972"
    },
    "tags": [
      {
        "description": "",
        "id": "f52d83ba-daa7-4d5c

              "num_gpus",
              "score"
            ],
            "type": "object"
          }
        },
        "required": [
          "machine_config",
          "benchmark",
          "training",
          "global_start_time",
          "global_end_time"
        ],
        "type": "object"
      },
      "upload_datetime": "2021-10-13T15:07:45.672697"
    },
    "execution_datetime": "2021-02-23T10:30:00",
    "flavor": {
      "description": "8x NVIDIA A100-40",
      "id": "b91268ca-9c97-4b56-85d4-b83f17123322",
      "name": "haicore-gpu8",
      "upload_datetime": "2021-10-13T15:35:04.713245"
    },
    "id": "5736c4f3-ea86-477a-8ae9-0e0ae05f62a3",
    "json": {
      "benchmark": {
        "dataset": "imagenet_mini",
        "docker_base_image": "nvcr.io/nvidia/tensorflow:20.06-tf2-py3",
        "flavor": "dataset",
        "tf_version": "2.2.0",
        "version": "0.6.0"
      },
      "global_end_time": "2021-02-23T11:30:08.712483Z",
      "global_execution_time_s

...

                    "type": "number"
                  },
                  "num_cores": {
                    "type": "number"
                  }
                },
                "type": "object"
              },
              "gpu_info": {
                "properties": {
                  "count": {
                    "type": "number"
                  },
                  "model": {
                    "type": "string"
                  }
                },
                "type": "object"
              },
              "memory_available": {
                "type": "number"
              },
              "memory_total": {
                "type": "number"
              }
            },
            "type": "object"
          },
          "training": {
            "properties": {
              "device": {
                "suggestToUser": true,
                "type": "string"
              },
              "models": {
                "type": "array"
              },
              "

          "execution_time_sec": 52.93564200401306,
          "num_batches": 100,
          "num_epochs": 0
        },
        "score": 3558.302476003574,
        "use_fp16": false,
        "variable_update": "parameter_server",
        "vgg16": {
          "average_examples_per_sec": 380.6035307423708,
          "batch_size": 256,
          "batch_size_per_device": 128,
          "execution_time_sec": 66.57822585105896,
          "num_batches": 100,
          "num_epochs": 0
        }
      }
    },
    "site": {
      "address": "unknown",
      "id": "d1e5dbbd-896c-48f8-b168-ff224073f137",
      "name": "IFCA-LCG2",
      "upload_datetime": "2021-10-15T12:59:30.458617"
    },
    "tags": [
      {
        "description": "",
        "id": "f52d83ba-daa7-4d5c-899e-ae5d5c2487ad",
        "name": "gpu"
      }
    ],
    "upload_datetime": "2021-10-18T22:25:52.629319"
  },
  {
    "benchmark": {
      "description": "tf_cnn_benchmarks accessed via DEEP API. Several Convolutional Neural N

          "training",
          "global_start_time",
          "global_end_time"
        ],
        "type": "object"
      },
      "upload_datetime": "2021-10-13T15:07:45.672697"
    },
    "execution_datetime": "2021-03-02T20:24:00",
    "flavor": {
      "id": "5e5beb68-ece0-4835-ab84-074d3e66783b",
      "name": "eu.indigo-datacloud.marathon",
      "upload_datetime": "2021-10-18T22:21:13.744202"
    },
    "id": "51b861a3-d1fe-403e-b2db-fa8467a40c0b",
    "json": {
      "benchmark": {
        "dataset": "imagenet_mini",
        "docker_base_image": "nvcr.io/nvidia/tensorflow:20.06-tf2-py3",
        "flavor": "dataset",
        "tf_version": "2.2.0",
        "version": "0.6.0"
      },
      "global_end_time": "2021-03-02T21:24:48.233254Z",
      "global_execution_time_sec": 489.1426429748535,
      "global_start_time": "2021-03-02T21:16:39.090611Z",
      "machine_config": {
        "cpu_info": {
          "cpu_info": "Intel(R) Xeon(R) Gold 6230 CPU @ 2.10GHz",
          "mhz_per

              },
              "version": {
                "type": "string"
              }
            },
            "required": [
              "flavor",
              "dataset"
            ],
            "type": "object"
          },
          "global_end_time": {
            "type": "string"
          },
          "global_execution_time_sec": {
            "type": "number"
          },
          "global_start_time": {
            "type": "string"
          },
          "machine_config": {
            "properties": {
              "cpu_info": {
                "properties": {
                  "cpu_info": {
                    "type": "string"
                  },
                  "mhz_per_cpu": {
                    "type": "number"
                  },
                  "num_cores": {
                    "type": "number"
                  }
                },
                "type": "object"
              },
              "gpu_info": {
                "properties": {
          

          "num_batches": 100,
          "num_epochs": 0
        },
        "inception3": {
          "average_examples_per_sec": 472.9028587400651,
          "batch_size": 384,
          "batch_size_per_device": 192,
          "execution_time_sec": 80.37296509742737,
          "num_batches": 100,
          "num_epochs": 0
        },
        "local_parameter_device": "cpu",
        "models": [
          "googlenet",
          "inception3",
          "resnet50",
          "vgg16"
        ],
        "num_gpus": 2,
        "optimizer": "sgd",
        "resnet50": {
          "average_examples_per_sec": 719.6569019732058,
          "batch_size": 384,
          "batch_size_per_device": 192,
          "execution_time_sec": 52.83391284942627,
          "num_batches": 100,
          "num_epochs": 0
        },
        "score": 3506.5330104922095,
        "use_fp16": false,
        "variable_update": "parameter_server",
        "vgg16": {
          "average_examples_per_sec": 358.3857454156006,
  

#### Search for the site id used to run our benchmark
You can get a list of all available benchmarks using GET /sites

In [4]:
sites=$(curl -X 'GET' "$eosc_perf_api/sites?name=ForHLR2")
site=$(echo $sites | jq '.items[0]')
echo $site | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   378  100   378    0     0    989      0 --:--:-- --:--:-- --:--:--   989
{
  "address": "https://wiki.scc.kit.edu/hpc/index.php?title=Category:ForHLR",
  "description": "Baden-Württemberg's general purpose tier 2 high performance computing (HPC) cluster",
  "id": "27ee87ff-a9aa-4486-891a-aff8164a9cab",
  "name": "ForHLR2",
  "upload_datetime": "2021-10-13T15:11:17.437972"
}


In [5]:
site_id=$(echo $site | jq -r '.id')
curl -X 'GET' "$eosc_perf_api/results?site_id=$site_id" \
  -H 'accept: application/json' | jq '.items'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29491  100 29491    0     0  55330      0 --:--:-- --:--:-- --:--:-- 55226
[
  {
    "benchmark": {
      "description": "tf_cnn_benchmarks accessed via DEEP API. Several Convolutional Neural Networks are executed to evaluate the performance.",
      "docker_image": "deephdc/deep-oc-benchmarks_cnn",
      "docker_tag": "benchmark",
      "id": "5a0f5641-01ae-4f48-b5d1-84cd9c87dfac",
      "json_schema": {
        "$schema": "https://json-schema.org/draft/2020-12/schema",
        "properties": {
          "benchmark": {
            "properties": {
              "dataset": {
                "suggestToUser": true,
                "type": "string"
              },
              "flavor": {
                "suggestToUser": true,
                "type": "string"
              },
              "version": {
                "type": "st

      "global_start_time": "2021-02-23T11:02:35.696854Z",
      "machine_config": {
        "cpu_info": {
          "cpu_info": "AMD EPYC 7742 64-Core Processor",
          "mhz_per_cpu": 3317.532,
          "num_cores": 256
        },
        "gpu_info": {
          "count": 8,
          "memory": 39420913408,
          "model": "A100-SXM4-40GB"
        },
        "memory_available": 1060777299968,
        "memory_total": 1082004615168
      },
      "training": {
        "allow_growth": true,
        "data_format": "NCHW",
        "device": "gpu",
        "googlenet": {
          "average_examples_per_sec": 1684.1920130854724,
          "batch_size": 960,
          "batch_size_per_device": 960,
          "execution_time_sec": 56.42832899093628,
          "num_batches": 100,
          "num_epochs": 0
        },
        "inception3": {
          "average_examples_per_sec": 481.3027947576969,
          "batch_size": 240,
          "batch_size_per_device": 240,
          "execution_time_

                  },
                  "mhz_per_cpu": {
                    "type": "number"
                  },
                  "num_cores": {
                    "type": "number"
                  }
                },
                "type": "object"
              },
              "gpu_info": {
                "properties": {
                  "count": {
                    "type": "number"
                  },
                  "model": {
                    "type": "string"
                  }
                },
                "type": "object"
              },
              "memory_available": {
                "type": "number"
              },
              "memory_total": {
                "type": "number"
              }
            },
            "type": "object"
          },
          "training": {
            "properties": {
              "device": {
                "suggestToUser": true,
                "type": "string"
              },
              "models": {
        

          "batch_size": 240,
          "batch_size_per_device": 240,
          "execution_time_sec": 40.239657163619995,
          "num_batches": 100,
          "num_epochs": 0
        },
        "score": 2936.8554382911675,
        "use_fp16": false,
        "variable_update": "parameter_server",
        "vgg16": {
          "average_examples_per_sec": 309.50671192646723,
          "batch_size": 160,
          "batch_size_per_device": 160,
          "execution_time_sec": 51.15352201461792,
          "num_batches": 100,
          "num_epochs": 0
        }
      }
    },
    "site": {
      "address": "https://wiki.scc.kit.edu/hpc/index.php?title=Category:ForHLR",
      "description": "Baden-Württemberg's general purpose tier 2 high performance computing (HPC) cluster",
      "id": "27ee87ff-a9aa-4486-891a-aff8164a9cab",
      "name": "ForHLR2",
      "upload_datetime": "2021-10-13T15:11:17.437972"
    },
    "tags": [
      {
        "description": "",
        "id": "f52d83ba-daa7-4d5c

              "num_gpus",
              "score"
            ],
            "type": "object"
          }
        },
        "required": [
          "machine_config",
          "benchmark",
          "training",
          "global_start_time",
          "global_end_time"
        ],
        "type": "object"
      },
      "upload_datetime": "2021-10-13T15:07:45.672697"
    },
    "execution_datetime": "2021-02-23T10:30:00",
    "flavor": {
      "description": "8x NVIDIA A100-40",
      "id": "b91268ca-9c97-4b56-85d4-b83f17123322",
      "name": "haicore-gpu8",
      "upload_datetime": "2021-10-13T15:35:04.713245"
    },
    "id": "5736c4f3-ea86-477a-8ae9-0e0ae05f62a3",
    "json": {
      "benchmark": {
        "dataset": "imagenet_mini",
        "docker_base_image": "nvcr.io/nvidia/tensorflow:20.06-tf2-py3",
        "flavor": "dataset",
        "tf_version": "2.2.0",
        "version": "0.6.0"
      },
      "global_end_time": "2021-02-23T11:30:08.712483Z",
      "global_execution_time_s

...

      "id": "b91268ca-9c97-4b56-85d4-b83f17123322",
      "name": "haicore-gpu8",
      "upload_datetime": "2021-10-13T15:35:04.713245"
    },
    "id": "e79a99b4-46f1-4ace-8d27-bc5f254b7a42",
    "json": {
      "benchmark": {
        "dataset": "synthetic_data",
        "docker_base_image": "nvcr.io/nvidia/tensorflow:20.06-tf2-py3",
        "flavor": "synthetic",
        "tf_version": "2.2.0",
        "version": "0.6.0"
      },
      "global_end_time": "2021-02-23T12:13:42.310329Z",
      "global_execution_time_sec": 466.6345520019531,
      "global_start_time": "2021-02-23T12:05:55.675777Z",
      "machine_config": {
        "cpu_info": {
          "cpu_info": "AMD EPYC 7742 64-Core Processor",
          "mhz_per_cpu": 3314.572,
          "num_cores": 256
        },
        "gpu_info": {
          "count": 8,
          "memory": 39420913408,
          "model": "A100-SXM4-40GB"
        },
        "memory_available": 1062193074176,
        "memory_total": 1082004582400
      },
     

              "flavor",
              "dataset"
            ],
            "type": "object"
          },
          "global_end_time": {
            "type": "string"
          },
          "global_execution_time_sec": {
            "type": "number"
          },
          "global_start_time": {
            "type": "string"
          },
          "machine_config": {
            "properties": {
              "cpu_info": {
                "properties": {
                  "cpu_info": {
                    "type": "string"
                  },
                  "mhz_per_cpu": {
                    "type": "number"
                  },
                  "num_cores": {
                    "type": "number"
                  }
                },
                "type": "object"
              },
              "gpu_info": {
                "properties": {
                  "count": {
                    "type": "number"
                  },
                  "model": {
                    "type": 

          "average_examples_per_sec": 1316.101905705066,
          "batch_size": 960,
          "batch_size_per_device": 240,
          "execution_time_sec": 72.25684809684753,
          "num_batches": 100,
          "num_epochs": 0
        },
        "local_parameter_device": "cpu",
        "models": [
          "googlenet",
          "inception3",
          "resnet50",
          "vgg16"
        ],
        "num_gpus": 4,
        "optimizer": "sgd",
        "resnet50": {
          "average_examples_per_sec": 1607.2773590283105,
          "batch_size": 960,
          "batch_size_per_device": 240,
          "execution_time_sec": 59.14579105377197,
          "num_batches": 100,
          "num_epochs": 0
        },
        "score": 8531.297569976781,
        "use_fp16": false,
        "variable_update": "parameter_server",
        "vgg16": {
          "average_examples_per_sec": 642.5885371362209,
          "batch_size": 640,
          "batch_size_per_device": 160,
          "execution_tim

                "type": "string"
              },
              "models": {
                "type": "array"
              },
              "num_gpus": {
                "suggestToUser": true,
                "type": "number"
              },
              "optimizer": {
                "type": "string"
              },
              "score": {
                "suggestToUser": true,
                "type": "number"
              }
            },
            "required": [
              "device",
              "num_gpus",
              "score"
            ],
            "type": "object"
          }
        },
        "required": [
          "machine_config",
          "benchmark",
          "training",
          "global_start_time",
          "global_end_time"
        ],
        "type": "object"
      },
      "upload_datetime": "2021-10-13T15:07:45.672697"
    },
    "execution_datetime": "2021-02-23T11:30:00",
    "flavor": {
      "description": "8x NVIDIA A100-40",
      "id": "b91268

    "tags": [
      {
        "description": "",
        "id": "f52d83ba-daa7-4d5c-899e-ae5d5c2487ad",
        "name": "gpu"
      }
    ],
    "upload_datetime": "2021-10-18T22:00:40.807040"
  }
]


#### Search for the flavor id used to run our benchmark
If needed you can also get a list of all available flavors in that site using GET /sites{id}/flavors

In [6]:
site_id=$(echo $site | jq -r '.id')
flavors=$(curl -X 'GET' "$eosc_perf_api/sites/$site_id/flavors:search?terms=haicore-gpu8")
flavor=$(echo $flavors | jq '.items[0]')
echo $flavor | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   239  100   239    0     0    483      0 --:--:-- --:--:-- --:--:--   482
{
  "description": "8x NVIDIA A100-40",
  "id": "b91268ca-9c97-4b56-85d4-b83f17123322",
  "name": "haicore-gpu8",
  "upload_datetime": "2021-10-13T15:35:04.713245"
}


In [7]:
flavor_id=$(echo $flavor | jq -r '.id')
curl -X 'GET' "$eosc_perf_api/results?flavor_id=$site_id" \
  -H 'accept: application/json' | jq '.items'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    91  100    91    0     0    211      0 --:--:-- --:--:-- --:--:--   211
[]


#### Search for the tags to relate your result
Collect the tags you want to link to your result so users can find it easily. <br>
> If you do not know the name of the tag you can use :search as generic filter.

In [8]:
tag_gpu=$(curl -X 'GET' "$eosc_perf_api/tags?name=gpu" | jq '.items[0]')
echo $tag_gpu | jq
tag_hpc=$(curl -X 'GET' "$eosc_perf_api/tags?name=hpc" | jq '.items[0]')
echo $tag_hpc | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   166  100   166    0     0    553      0 --:--:-- --:--:-- --:--:--   553
{
  "description": "",
  "id": "f52d83ba-daa7-4d5c-899e-ae5d5c2487ad",
  "name": "gpu"
}
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    91  100    91    0     0    326      0 --:--:-- --:--:-- --:--:--   326
null


In [9]:
tag_1=$(echo $tag_gpu | jq -r '.id')
tag_2=$(echo $tag_hpc | jq -r '.id')
curl -X 'GET' "$eosc_perf_api/results?tag_id=$tag_1&tag_id=$tag_2" \
  -H 'accept: application/json' | jq '.items'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    94  100    94    0     0    482      0 --:--:-- --:--:-- --:--:--   482
null


#### Search for results between dates
If is also possible to filter results by upload and execution date.

In [10]:
upload_before='2021-10-16'
execution_after='2021-10-14'

curl -X 'GET' "$eosc_perf_api/results?upload_before=$upload_before&execution_after=$execution_after" \
  -H 'accept: application/json' | jq '.items'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 71387  100 71387    0     0   233k      0 --:--:-- --:--:-- --:--:--  233k
[
  {
    "benchmark": {
      "description": "Compare cpu perf with multithreaded raytracing",
      "docker_image": "thechristophe/openbench-c-ray",
      "docker_tag": "latest",
      "id": "8edf5bcf-d432-4f4f-930f-6245d7f49588",
      "json_schema": {
        "$schema": "https://json-schema.org/draft/2020-12/schema",
        "properties": {
          "arguments": {
            "const": "Total Time - 4K, 16 Rays Per Pixel"
          },
          "machine": {
            "properties": {
              "cpu": {
                "properties": {
                  "arch": {
                    "suggestToUser": true,
                    "type": "string"
                  },
                  "bits": {
                    "type": "number"
                  },

            "rdrnd",
            "rdseed",
            "rdtscp",
            "rep_good",
            "sep",
            "sse",
            "sse2",
            "sse4_1",
            "sse4_2",
            "sse4a",
            "ssse3",
            "syscall",
            "tsc",
            "tsc_known_freq",
            "vme",
            "vmmcall",
            "x2apic",
            "xsave"
          ],
          "hz_actual": [
            3700000000,
            0
          ],
          "hz_actual_friendly": "3.7000 GHz",
          "hz_advertised": [
            3700000000,
            0
          ],
          "hz_advertised_friendly": "3.7000 GHz",
          "l1_data_cache_size": "32 KiB",
          "l1_instruction_cache_size": "32 KiB",
          "l2_cache_associativity": 6,
          "l2_cache_line_size": 512,
          "l2_cache_size": "512 KiB",
          "l3_cache_size": 524288,
          "model": 33,
          "python_version": "3.8.5.final.0 (64 bit)",
          "vendor_id_raw": "A

      "machine": {
        "cpu": {
          "arch": "X86_64",
          "arch_string_raw": "x86_64",
          "bits": 64,
          "brand_raw": "AMD Ryzen 9 5900X 12-Core Processor",
          "count": 2,
          "cpuinfo_version": [
            7,
            0,
            0
          ],
          "cpuinfo_version_string": "7.0.0",
          "family": 25,
          "flags": [
            "3dnowprefetch",
            "abm",
            "aes",
            "apic",
            "arat",
            "avx",
            "avx2",
            "clflush",
            "clflushopt",
            "cmov",
            "cmp_legacy",
            "constant_tsc",
            "cpuid",
            "cr8_legacy",
            "cx16",
            "cx8",
            "de",
            "extd_apicid",
            "fpu",
            "fsgsbase",
            "fxsr",
            "fxsr_opt",
            "ht",
            "hypervisor",
            "invpcid",
            "lahf_lm",
            "lm",
            "mca",

                ],
                "type": "object"
              }
            },
            "type": "object"
          },
          "result": {
            "description": "The execution results",
            "properties": {
              "all_results": {
                "type": "string"
              },
              "score": {
                "suggestToUser": true,
                "type": "number"
              }
            },
            "required": [
              "all_results",
              "score"
            ],
            "type": "object"
          },
          "test": {
            "const": "pts/c-ray-1.2.0"
          },
          "units": {
            "const": "Seconds"
          }
        },
        "required": [
          "arguments",
          "machine",
          "result",
          "test",
          "units"
        ],
        "type": "object"
      },
      "upload_datetime": "2021-10-15T13:00:14.658514"
    },
    "execution_datetime": "2021-10-15T15:00:18.400299",

      "id": "f00e335a-158e-457d-801d-133f19e7d126",
      "name": "VirtualboxTestbed",
      "upload_datetime": "2021-10-15T12:59:03.958477"
    },
    "tags": [],
    "upload_datetime": "2021-10-15T13:00:19.187816"
  },
  {
    "benchmark": {
      "description": "Compare cpu perf with multithreaded raytracing",
      "docker_image": "thechristophe/openbench-c-ray",
      "docker_tag": "latest",
      "id": "8edf5bcf-d432-4f4f-930f-6245d7f49588",
      "json_schema": {
        "$schema": "https://json-schema.org/draft/2020-12/schema",
        "properties": {
          "arguments": {
            "const": "Total Time - 4K, 16 Rays Per Pixel"
          },
          "machine": {
            "properties": {
              "cpu": {
                "properties": {
                  "arch": {
                    "suggestToUser": true,
                    "type": "string"
                  },
                  "bits": {
                    "type": "number"
                  },
                 

...

    "id": "d9bac185-2b99-4e26-8176-3619ebb6a6cc",
    "json": {
      "arguments": "Total Time - 4K, 16 Rays Per Pixel",
      "machine": {
        "cpu": {
          "arch": "X86_64",
          "arch_string_raw": "x86_64",
          "bits": 64,
          "brand_raw": "Intel(R) Xeon(R) CPU E5-2670 0 @ 2.60GHz",
          "count": 2,
          "cpuinfo_version": [
            7,
            0,
            0
          ],
          "cpuinfo_version_string": "7.0.0",
          "family": 6,
          "flags": [
            "aes",
            "apic",
            "arat",
            "arch_perfmon",
            "avx",
            "clflush",
            "cmov",
            "constant_tsc",
            "cpuid",
            "cpuid_fault",
            "cx16",
            "cx8",
            "de",
            "ept",
            "flexpriority",
            "fpu",
            "fxsr",
            "hypervisor",
            "ibpb",
            "ibrs",
            "lahf_lm",
            "lm",
            "

                    "type": "string"
                  }
                },
                "required": [
                  "count"
                ],
                "type": "object"
              }
            },
            "type": "object"
          },
          "result": {
            "description": "The execution results",
            "properties": {
              "all_results": {
                "type": "string"
              },
              "score": {
                "suggestToUser": true,
                "type": "number"
              }
            },
            "required": [
              "all_results",
              "score"
            ],
            "type": "object"
          },
          "test": {
            "const": "pts/c-ray-1.2.0"
          },
          "units": {
            "const": "Seconds"
          }
        },
        "required": [
          "arguments",
          "machine",
          "result",
          "test",
          "units"
        ],
        "type": "o

      "units": "Seconds"
    },
    "site": {
      "address": "unknown",
      "id": "d1e5dbbd-896c-48f8-b168-ff224073f137",
      "name": "IFCA-LCG2",
      "upload_datetime": "2021-10-15T12:59:30.458617"
    },
    "tags": [],
    "upload_datetime": "2021-10-15T13:00:27.672467"
  },
  {
    "benchmark": {
      "description": "Compare cpu perf with multithreaded raytracing",
      "docker_image": "thechristophe/openbench-c-ray",
      "docker_tag": "latest",
      "id": "8edf5bcf-d432-4f4f-930f-6245d7f49588",
      "json_schema": {
        "$schema": "https://json-schema.org/draft/2020-12/schema",
        "properties": {
          "arguments": {
            "const": "Total Time - 4K, 16 Rays Per Pixel"
          },
          "machine": {
            "properties": {
              "cpu": {
                "properties": {
                  "arch": {
                    "suggestToUser": true,
                    "type": "string"
                  },
                  "bits": {
         

            "pni",
            "popcnt",
            "pse",
            "pse36",
            "pti",
            "rdtscp",
            "rep_good",
            "sep",
            "ss",
            "ssbd",
            "sse",
            "sse2",
            "sse4_1",
            "sse4_2",
            "ssse3",
            "syscall",
            "tpr_shadow",
            "tsc",
            "tsc_adjust",
            "tsc_deadline_timer",
            "tscdeadline",
            "vme",
            "vmx",
            "vnmi",
            "vpid",
            "x2apic",
            "xsave",
            "xsaveopt",
            "xtopology"
          ],
          "hz_actual": [
            2600027000,
            0
          ],
          "hz_actual_friendly": "2.6000 GHz",
          "hz_advertised": [
            2600000000,
            0
          ],
          "hz_advertised_friendly": "2.6000 GHz",
          "l1_data_cache_size": "256 KiB",
          "l1_instruction_cache_size": "256 KiB",
          "

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#### Search using custom filters
If is also possible to filter results specific values inside the result (if you know the result structure).
> If you do not want results that does not include the field in the response, we recommend that you use a benchmark_id

In [11]:
benchmark_id='5a0f5641-01ae-4f48-b5d1-84cd9c87dfac'
filter_1='machine_config.gpu_info.count%20%3E%204'
filter_2='machine_config.gpu_info.count%20%3C%2020'

curl -X 'GET' "$eosc_perf_api/results?benchmark_id=$benchmark_id&filters=$filter_1&filters=$filter_2" \
  -H 'accept: application/json' | jq '.items'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29491  100 29491    0     0   119k      0 --:--:-- --:--:-- --:--:--  119k
[
  {
    "benchmark": {
      "description": "tf_cnn_benchmarks accessed via DEEP API. Several Convolutional Neural Networks are executed to evaluate the performance.",
      "docker_image": "deephdc/deep-oc-benchmarks_cnn",
      "docker_tag": "benchmark",
      "id": "5a0f5641-01ae-4f48-b5d1-84cd9c87dfac",
      "json_schema": {
        "$schema": "https://json-schema.org/draft/2020-12/schema",
        "properties": {
          "benchmark": {
            "properties": {
              "dataset": {
                "suggestToUser": true,
                "type": "string"
              },
              "flavor": {
                "suggestToUser": true,
                "type": "string"
              },
              "version": {
                "type": "st

      "global_start_time": "2021-02-23T11:02:35.696854Z",
      "machine_config": {
        "cpu_info": {
          "cpu_info": "AMD EPYC 7742 64-Core Processor",
          "mhz_per_cpu": 3317.532,
          "num_cores": 256
        },
        "gpu_info": {
          "count": 8,
          "memory": 39420913408,
          "model": "A100-SXM4-40GB"
        },
        "memory_available": 1060777299968,
        "memory_total": 1082004615168
      },
      "training": {
        "allow_growth": true,
        "data_format": "NCHW",
        "device": "gpu",
        "googlenet": {
          "average_examples_per_sec": 1684.1920130854724,
          "batch_size": 960,
          "batch_size_per_device": 960,
          "execution_time_sec": 56.42832899093628,
          "num_batches": 100,
          "num_epochs": 0
        },
        "inception3": {
          "average_examples_per_sec": 481.3027947576969,
          "batch_size": 240,
          "batch_size_per_device": 240,
          "execution_time_

                  },
                  "mhz_per_cpu": {
                    "type": "number"
                  },
                  "num_cores": {
                    "type": "number"
                  }
                },
                "type": "object"
              },
              "gpu_info": {
                "properties": {
                  "count": {
                    "type": "number"
                  },
                  "model": {
                    "type": "string"
                  }
                },
                "type": "object"
              },
              "memory_available": {
                "type": "number"
              },
              "memory_total": {
                "type": "number"
              }
            },
            "type": "object"
          },
          "training": {
            "properties": {
              "device": {
                "suggestToUser": true,
                "type": "string"
              },
              "models": {
        

          "batch_size": 240,
          "batch_size_per_device": 240,
          "execution_time_sec": 40.239657163619995,
          "num_batches": 100,
          "num_epochs": 0
        },
        "score": 2936.8554382911675,
        "use_fp16": false,
        "variable_update": "parameter_server",
        "vgg16": {
          "average_examples_per_sec": 309.50671192646723,
          "batch_size": 160,
          "batch_size_per_device": 160,
          "execution_time_sec": 51.15352201461792,
          "num_batches": 100,
          "num_epochs": 0
        }
      }
    },
    "site": {
      "address": "https://wiki.scc.kit.edu/hpc/index.php?title=Category:ForHLR",
      "description": "Baden-Württemberg's general purpose tier 2 high performance computing (HPC) cluster",
      "id": "27ee87ff-a9aa-4486-891a-aff8164a9cab",
      "name": "ForHLR2",
      "upload_datetime": "2021-10-13T15:11:17.437972"
    },
    "tags": [
      {
        "description": "",
        "id": "f52d83ba-daa7-4d5c

              "num_gpus",
              "score"
            ],
            "type": "object"
          }
        },
        "required": [
          "machine_config",
          "benchmark",
          "training",
          "global_start_time",
          "global_end_time"
        ],
        "type": "object"
      },
      "upload_datetime": "2021-10-13T15:07:45.672697"
    },
    "execution_datetime": "2021-02-23T10:30:00",
    "flavor": {
      "description": "8x NVIDIA A100-40",
      "id": "b91268ca-9c97-4b56-85d4-b83f17123322",
      "name": "haicore-gpu8",
      "upload_datetime": "2021-10-13T15:35:04.713245"
    },
    "id": "5736c4f3-ea86-477a-8ae9-0e0ae05f62a3",
    "json": {
      "benchmark": {
        "dataset": "imagenet_mini",
        "docker_base_image": "nvcr.io/nvidia/tensorflow:20.06-tf2-py3",
        "flavor": "dataset",
        "tf_version": "2.2.0",
        "version": "0.6.0"
      },
      "global_end_time": "2021-02-23T11:30:08.712483Z",
      "global_execution_time_s

...

      "id": "b91268ca-9c97-4b56-85d4-b83f17123322",
      "name": "haicore-gpu8",
      "upload_datetime": "2021-10-13T15:35:04.713245"
    },
    "id": "e79a99b4-46f1-4ace-8d27-bc5f254b7a42",
    "json": {
      "benchmark": {
        "dataset": "synthetic_data",
        "docker_base_image": "nvcr.io/nvidia/tensorflow:20.06-tf2-py3",
        "flavor": "synthetic",
        "tf_version": "2.2.0",
        "version": "0.6.0"
      },
      "global_end_time": "2021-02-23T12:13:42.310329Z",
      "global_execution_time_sec": 466.6345520019531,
      "global_start_time": "2021-02-23T12:05:55.675777Z",
      "machine_config": {
        "cpu_info": {
          "cpu_info": "AMD EPYC 7742 64-Core Processor",
          "mhz_per_cpu": 3314.572,
          "num_cores": 256
        },
        "gpu_info": {
          "count": 8,
          "memory": 39420913408,
          "model": "A100-SXM4-40GB"
        },
        "memory_available": 1062193074176,
        "memory_total": 1082004582400
      },
     

              "flavor",
              "dataset"
            ],
            "type": "object"
          },
          "global_end_time": {
            "type": "string"
          },
          "global_execution_time_sec": {
            "type": "number"
          },
          "global_start_time": {
            "type": "string"
          },
          "machine_config": {
            "properties": {
              "cpu_info": {
                "properties": {
                  "cpu_info": {
                    "type": "string"
                  },
                  "mhz_per_cpu": {
                    "type": "number"
                  },
                  "num_cores": {
                    "type": "number"
                  }
                },
                "type": "object"
              },
              "gpu_info": {
                "properties": {
                  "count": {
                    "type": "number"
                  },
                  "model": {
                    "type": 

          "average_examples_per_sec": 1316.101905705066,
          "batch_size": 960,
          "batch_size_per_device": 240,
          "execution_time_sec": 72.25684809684753,
          "num_batches": 100,
          "num_epochs": 0
        },
        "local_parameter_device": "cpu",
        "models": [
          "googlenet",
          "inception3",
          "resnet50",
          "vgg16"
        ],
        "num_gpus": 4,
        "optimizer": "sgd",
        "resnet50": {
          "average_examples_per_sec": 1607.2773590283105,
          "batch_size": 960,
          "batch_size_per_device": 240,
          "execution_time_sec": 59.14579105377197,
          "num_batches": 100,
          "num_epochs": 0
        },
        "score": 8531.297569976781,
        "use_fp16": false,
        "variable_update": "parameter_server",
        "vgg16": {
          "average_examples_per_sec": 642.5885371362209,
          "batch_size": 640,
          "batch_size_per_device": 160,
          "execution_tim

                "type": "string"
              },
              "models": {
                "type": "array"
              },
              "num_gpus": {
                "suggestToUser": true,
                "type": "number"
              },
              "optimizer": {
                "type": "string"
              },
              "score": {
                "suggestToUser": true,
                "type": "number"
              }
            },
            "required": [
              "device",
              "num_gpus",
              "score"
            ],
            "type": "object"
          }
        },
        "required": [
          "machine_config",
          "benchmark",
          "training",
          "global_start_time",
          "global_end_time"
        ],
        "type": "object"
      },
      "upload_datetime": "2021-10-13T15:07:45.672697"
    },
    "execution_datetime": "2021-02-23T11:30:00",
    "flavor": {
      "description": "8x NVIDIA A100-40",
      "id": "b91268

    "tags": [
      {
        "description": "",
        "id": "f52d83ba-daa7-4d5c-899e-ae5d5c2487ad",
        "name": "gpu"
      }
    ],
    "upload_datetime": "2021-10-18T22:00:40.807040"
  }
]


#### Configure sorting and use pagination
For all the previous options, it is possible to sort the results using the following fields:
- "id": Result id.
- "upload_datetime": EOSC Performance upload datetime of the result.
- "json": Result json values.
- "execution_datetime": Execution date of the benchmark.
- "benchmark_id": Benchmark id used to obtain the result.
- "flavor_id": Favor id used to run the benchmark.
- "site_id": Site id where the benchmark was executed.

In addition, sometimes you might get more results that expected. In such case you will have to use pagination to collect all the items. To do so you can use the following parameters:
- per_page: The number of items to be displayed on a page (maximum 100)
- page: The return page number (1 indexed)

In [12]:
curl -X 'GET' "$eosc_perf_api/results?per_page=4&page=2&sort_by=%2Bid" \
  -H 'accept: application/json' | jq '.items'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12579  100 12579    0     0  78130      0 --:--:-- --:--:-- --:--:-- 78130
[
  {
    "benchmark": {
      "description": "tf_cnn_benchmarks accessed via DEEP API. Several Convolutional Neural Networks are executed to evaluate the performance.",
      "docker_image": "deephdc/deep-oc-benchmarks_cnn",
      "docker_tag": "benchmark",
      "id": "5a0f5641-01ae-4f48-b5d1-84cd9c87dfac",
      "json_schema": {
        "$schema": "https://json-schema.org/draft/2020-12/schema",
        "properties": {
          "benchmark": {
            "properties": {
              "dataset": {
                "suggestToUser": true,
                "type": "string"
              },
              "flavor": {
                "suggestToUser": true,
                "type": "string"
              },
              "version": {
                "type": "st

      "global_start_time": "2021-02-23T12:04:19.674702Z",
      "machine_config": {
        "cpu_info": {
          "cpu_info": "AMD EPYC 7742 64-Core Processor",
          "mhz_per_cpu": 3291.864,
          "num_cores": 256
        },
        "gpu_info": {
          "count": 8,
          "memory": 39420913408,
          "model": "A100-SXM4-40GB"
        },
        "memory_available": 1061119328256,
        "memory_total": 1082004615168
      },
      "training": {
        "allow_growth": true,
        "data_format": "NCHW",
        "device": "gpu",
        "googlenet": {
          "average_examples_per_sec": 3296.587519772614,
          "batch_size": 1920,
          "batch_size_per_device": 960,
          "execution_time_sec": 57.65860414505005,
          "num_batches": 100,
          "num_epochs": 0
        },
        "inception3": {
          "average_examples_per_sec": 946.5761978457389,
          "batch_size": 480,
          "batch_size_per_device": 240,
          "execution_time_

            "properties": {
              "all_results": {
                "type": "string"
              },
              "score": {
                "suggestToUser": true,
                "type": "number"
              }
            },
            "required": [
              "all_results",
              "score"
            ],
            "type": "object"
          },
          "test": {
            "const": "pts/c-ray-1.2.0"
          },
          "units": {
            "const": "Seconds"
          }
        },
        "required": [
          "arguments",
          "machine",
          "result",
          "test",
          "units"
        ],
        "type": "object"
      },
      "upload_datetime": "2021-10-15T13:00:14.658514"
    },
    "execution_datetime": "2021-10-15T15:00:21.644672",
    "flavor": {
      "description": "",
      "id": "87a41392-e3ea-4a19-b887-4096ddb875c6",
      "name": "standard.small",
      "upload_datetime": "2021-10-15T12:59:20.762669"
    },
    "id": "3

          "stepping": 4,
          "vendor_id_raw": "GenuineIntel"
        }
      },
      "result": {
        "all_results": "1015.953:1016.965:1016.778",
        "score": 1016.565
      },
      "test": "pts/c-ray-1.2.0",
      "units": "Seconds"
    },
    "site": {
      "address": "unknown",
      "id": "7858ccf2-fecc-4a08-bcd6-12e11c987774",
      "name": "CESNET-MCC",
      "upload_datetime": "2021-10-15T12:59:18.378423"
    },
    "tags": [],
    "upload_datetime": "2021-10-15T13:00:22.783940"
  },
  {
    "benchmark": {
      "description": "Compare cpu perf with multithreaded raytracing",
      "docker_image": "thechristophe/openbench-c-ray",
      "docker_tag": "latest",
      "id": "8edf5bcf-d432-4f4f-930f-6245d7f49588",
      "json_schema": {
        "$schema": "https://json-schema.org/draft/2020-12/schema",
        "properties": {
          "arguments": {
            "const": "Total Time - 4K, 16 Rays Per Pixel"
          },
          "machine": {
            "properties

            "fxsr",
            "fxsr_opt",
            "hypervisor",
            "ibpb",
            "lahf_lm",
            "lm",
            "mca",
            "mce",
            "misalignsse",
            "mmx",
            "mmxext",
            "movbe",
            "msr",
            "mtrr",
            "nopl",
            "nx",
            "osvw",
            "osxsave",
            "pae",
            "pat",
            "pclmulqdq",
            "pdpe1gb",
            "pge",
            "pni",
            "popcnt",
            "pse",
            "pse36",
            "rdrand",
            "rdrnd",
            "rdseed",
            "rdtscp",
            "rep_good",
            "sep",
            "sha",
            "sha_ni",
            "smap",
            "smep",
            "ssbd",
            "sse",
            "sse2",
            "sse4_1",
            "sse4_2",
            "sse4a",
            "ssse3",
            "syscall",
            "topoext",
            "tsc",
            "ts

              "all_results",
              "score"
            ],
            "type": "object"
          },
          "test": {
            "const": "pts/c-ray-1.2.0"
          },
          "units": {
            "const": "Seconds"
          }
        },
        "required": [
          "arguments",
          "machine",
          "result",
          "test",
          "units"
        ],
        "type": "object"
      },
      "upload_datetime": "2021-10-15T13:00:14.658514"
    },
    "execution_datetime": "2021-10-15T15:00:17.551684",
    "flavor": {
      "description": "",
      "id": "9600c29b-e53d-4025-bb41-cf07f5428e38",
      "name": "virtualbox-arch",
      "upload_datetime": "2021-10-15T12:59:06.452280"
    },
    "id": "42dda424-5ff1-4951-92e9-4c4c69d4f88f",
    "json": {
      "arguments": "Total Time - 4K, 16 Rays Per Pixel",
      "machine": {
        "cpu": {
          "arch": "X86_64",
          "arch_string_raw": "x86_64",
          "bits": 64,
          "brand_raw": "AMD 